In [81]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from keras.models import Sequential
from keras.layers import Dense
pd.options.mode.chained_assignment = None  # default='warn'


# pairnoume ta aparaitita arxeia 
data = pd.read_csv('Updated-Ratings.csv')
books = pd.read_csv('BX-Books.csv')
clust_1 = pd.read_csv('Clust_1.csv')
clust_2 = pd.read_csv('Clust_2.csv')
clust_3 = pd.read_csv('Clust_3.csv')

zero_ratings = data[data['rating'] == 0]

print('ARXIKA RATINGS: ', len(data))
print('ZERO RATINGS: ', len(zero_ratings), '\n')

c1_uids = list(set(clust_1.uid).intersection(set(data.uid)))
c2_uids = list(set(clust_2.uid).intersection(set(data.uid)))
c3_uids = list(set(clust_3.uid).intersection(set(data.uid)))


# diaxorizoume ta ratings pou exoume analoga me to cluster twn users pou tis ekanan
c1_ratings = data[data.uid.isin(c1_uids)]
c2_ratings = data[data.uid.isin(c2_uids)]
c3_ratings = data[data.uid.isin(c3_uids)]

#print(len(c1_ratings))
#print(len(c2_ratings))
#print(len(c3_ratings))

# apo edw argotera tha to automatopoihsoyme gia na ginetai mia fora gia ola.
clusters = [c1_ratings, c2_ratings, c3_ratings]

for cluster in clusters:
    # diaxwrizoume tis vathmologies tou kathe cluster se zero kai non_zero
    zero_ratings_clust = cluster[cluster['rating'] == 0]
    non_zero_ratings_clust = cluster[cluster['rating'] != 0]

    # prosthetoume sta dedomena ta summaries gia to kathe vivlio poy exei vathmologithei sto cluster
    sum1_non_zero = pd.merge(non_zero_ratings_clust, books, how='inner', on='isbn')
    sum1_zero = pd.merge(zero_ratings_clust, books, how='inner', on='isbn')

    # ksexwriszoume ta summaries se listes wste na einai etoima gia to vectorization
    sums = sum1_non_zero.summary.values.tolist()
    zero_sums = sum1_zero.summary.values.tolist()

    #makings vectors
    # gia ta non zero summaries pou tha einai to X_train
    print('Gia to trehon Cluster exoume: ' )

    model_vect = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    X_train = model_vect.encode(sums)
    print('Non zero ratings tou cluster: ', len(X_train))

    # oi vathmologies twn non zero books pou tha perasoun sto model mazi me ta summaries
    y_train = sum1_non_zero.rating.values.tolist()

    # gia ta zero summaries pou tha einai to X_test
    model_vect = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    X_test = model_vect.encode(zero_sums)
    print('Zero ratings tou cluster: ', len(X_test))

    # metatroph se np arrays giati allios den mporoun na mpoun sto keras model
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)

    #creating layers
    model = Sequential()
    model.add(Dense(384, input_shape = (384,)))
    #model.add(Dropout(0.3))
    model.add(Dense(192, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(96, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation='relu'))


    # model compile apairaitito prin tin xrhsh tou model 
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])


    model.fit(X_train, y_train, verbose = 0)

    y_pred = model.predict(X_test)

    # etoimasia twn ratings gia eisagwgh
    y_pred = np.array(y_pred)
    values = y_pred.flatten()
    values = np.round(values, 0)

    zero_ratings_clust['rating'] = values


    # eidagwgh twn provlepsewn sta arxika ratings kai antikatastash twn 0 
    data.update(zero_ratings_clust)

    zero_ratings = data[data['rating'] == 0]
    print('OLA TA ZERO RATINGS meta tin prosthiki tis provlepsis: ' , len(zero_ratings), '\n')






ARXIKA RATINGS:  42708
ZERO RATINGS:  13165 

Gia to trehon Cluster exoume: 
Non zero ratings tou cluster:  6835
Zero ratings tou cluster:  2079
65/65 [==============================] - 0s 1ms/step
OLA TA ZERO RATINGS meta tin prosthiki tis provlepsis:  11086 

Gia to trehon Cluster exoume: 
Non zero ratings tou cluster:  9812
Zero ratings tou cluster:  3160
99/99 [==============================] - 0s 1ms/step
OLA TA ZERO RATINGS meta tin prosthiki tis provlepsis:  7926 

Gia to trehon Cluster exoume: 
Non zero ratings tou cluster:  264
Zero ratings tou cluster:  97
4/4 [==============================] - 0s 2ms/step
OLA TA ZERO RATINGS meta tin prosthiki tis provlepsis:  7831 



In [82]:
print(data)

       Unnamed: 0       uid        isbn  rating
0             0.0  123629.0  0002005018     9.0
1             1.0  198013.0  0399135782     7.0
2             2.0  212449.0  0399135782     8.0
3             3.0  252921.0  0399135782     8.0
4             4.0   12923.0  0440234743    10.0
...           ...       ...         ...     ...
42703     42703.0  277831.0  0786811358     5.0
42704     42704.0  277834.0  0847817741     7.0
42705     42705.0  278714.0  0896101932     0.0
42706     42706.0  278789.0  1413702783     9.0
42707     42707.0  278849.0  0920656307     6.0

[42708 rows x 4 columns]
